<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-31 14:17:32
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20


-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  4.51 L
Current:  1.38 C
-------------------
Today PnL: -13.63 K (-0.1%)
Current PnL: -17.05 L (-11.6%)
CY Booked + Current PnL: -9.02 L (-6.13%)
-------------------
Total profit:  3.44 L
Total loss:  -20.49 L
-------------------
Total Booked + Current PnL: 17.96 L (14.96%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.84%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 75.28 L (54.72%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 9.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,BRITANNIA,5190.70,6446.05,16.68,H-LC,9.70,104715.0,11282.0,11309.0,1.23,12.08,10.80,24.18,31.0,1.00,0.79,27.15,XY25,ATH,FMCG
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110200.0,5888.0,11648.0,-0.01,5.64,10.57,16.81,10.0,0.51,0.83,22.60,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-17.63,H-LC,6.58,112170.0,8604.0,12103.0,0.38,8.31,10.79,20.00,15.0,0.71,0.84,23.55,X40N,BTT,HEALTHCARE
74,SIS,477.00,477.00,2346.11,H-SC,6.62,65816.0,-15274.0,15276.0,0.77,-18.84,23.21,0.00,157.0,-1.00,0.49,33.39,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-12.46,M-LC,4.29,212066.0,10586.0,16626.0,-0.17,5.25,7.84,13.51,62.0,0.64,1.59,18.51,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.44,M-SC,41.81,65251.0,245.0,150684.0,0.05,0.38,230.93,232.18,199.0,0.00,0.49,1.38,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.76,100452.0,-511.0,95038.0,3.33,-0.51,94.61,93.63,149.0,-0.01,0.75,49.62,SR,ATH,CHEMICALS
38,HEROMOTOCO,4311.81,5949.07,-6.67,H-MC,3.54,150150.0,-763.0,58063.0,0.91,-0.51,38.67,37.97,90.0,-0.01,1.13,24.67,AR,ATH,AUTO


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-18.76,H-LC,1.76,213967.0,9467.0,31945.0,0.15,4.63,14.93,20.25,9.0,0.30,1.61,13.77,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.0,-9.60,H-LC,2.47,187583.0,-12923.0,61696.0,-0.54,-6.45,32.89,24.33,30.0,-0.21,1.41,8.17,X40N,BTT,FINANCE
56,LTIM,5564.16,7230.2,-4.50,H-LC,2.69,183834.0,-16476.0,76457.0,-0.64,-8.23,41.59,29.94,24.0,-0.22,1.38,27.00,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-17.81,H-LC,3.24,163370.0,-38016.0,125632.0,-0.55,-18.88,76.90,43.51,19.0,-0.30,1.23,13.72,X40,BTT,PAINTS
40,HINDUNILVR,2413.81,2723.0,-9.09,H-LC,4.28,284816.0,14469.0,20165.0,4.33,5.35,7.08,12.81,28.0,0.72,2.14,19.12,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,HINDUNILVR,2413.81,2723.00,-9.09,H-LC,4.28,284816.0,14469.0,20165.0,4.33,5.35,7.08,12.81,28.0,0.72,2.14,19.12,XY25,NTT,FMCG
68,SAMMAANCAP,170.35,326.00,-167.05,M-SC,4.23,77070.0,-25140.0,118534.0,3.69,-24.60,153.80,91.37,210.0,-0.21,0.58,25.11,XY25,NTT,FINANCE
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.76,100452.0,-511.0,95038.0,3.33,-0.51,94.61,93.63,149.0,-0.01,0.75,49.62,SR,ATH,CHEMICALS
51,JIOFIN,310.58,387.00,8.02,H-LC,13.32,231490.0,14084.0,39400.0,3.25,6.48,17.02,24.61,50.0,0.36,1.74,64.59,XY24,BTT,FINANCE
88,UNITDSPR,1376.96,1644.00,-11.95,H-LC,7.42,177276.0,-4483.0,39728.0,2.07,-2.47,22.41,19.39,11.0,-0.11,1.33,4.85,X40N,NTT,BREWERIES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,223.00,M-SC,8.39,144277.0,-38801.0,117773.0,-7.60,-21.19,81.63,43.14,232.0,-0.33,1.08,35.71,XY24,NTT,MISC
64,RELAXO,902.64,1176.00,-34.93,H-SC,13.73,58816.0,-56722.0,91712.0,-4.86,-49.09,155.93,30.28,134.0,-0.62,0.44,14.90,X40N,NTT,FOOTWEAR
18,BSOFT,628.85,836.99,-16.31,H-SC,12.44,70158.0,-39891.0,76318.0,-3.34,-36.25,108.78,33.10,129.0,-0.52,0.53,16.58,XR,ATH,IT
93,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,7.51,100500.0,9002.0,69747.0,-3.18,9.84,69.40,86.07,211.0,0.13,0.76,45.38,XR,NTT,DURABLES
20,CAMS,3643.00,5226.82,-5.92,H-SC,0.23,105689.0,3685.0,40659.0,-2.97,3.61,38.47,43.48,120.0,0.09,0.79,21.00,X40N,ATH,MISC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-30.89,M-SC,8.87,147349.0,-23792.0,23738.0,-1.33,-13.90,16.11,-0.03,197.0,-1.00,1.11,6.63,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,6.15,148936.0,-25623.0,25662.0,-0.14,-14.68,17.23,0.02,135.0,-1.00,1.12,29.15,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.87,119259.0,-23220.0,23840.0,1.21,-16.30,19.99,0.44,163.0,-0.97,0.90,40.76,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-18.32,H-MC,1.83,85820.0,-16832.0,16829.0,0.48,-16.40,19.61,-0.00,101.0,-1.00,0.64,30.46,OX40N,NTT,IT
21,CERA,7989.07,8422.0,-17.18,H-SC,2.91,118494.0,-25309.0,33107.0,1.07,-17.60,27.94,5.42,155.0,-0.76,0.89,30.76,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.06,H-LC,13.42,238072.0,-57862.0,151771.0,-0.05,-19.55,63.75,31.73,1.0,-0.38,1.79,0.00,X40,BTT,IT
32,ENRIN,1377.95,3247.20,NaN,NaN,2.91,81180.0,46731.0,0.0,0.63,135.65,0.00,135.65,2.0,inf,0.61,24.14,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-7.17,H-LC,11.92,281751.0,16711.0,79933.0,0.31,6.30,28.37,36.46,3.0,0.21,2.12,20.33,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-14.93,H-LC,7.09,272331.0,10730.0,134886.0,0.16,4.10,49.53,55.67,4.0,0.08,2.05,10.46,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.48,H-LC,7.49,215865.0,-654.0,93556.0,-0.34,-0.30,43.34,42.91,5.0,-0.01,1.62,12.83,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.18,51612.0,10357.0,7076.0,-0.67,25.10,13.71,42.26,198.0,1.46,0.39,21.51,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,19.36,H-SC,12.11,152958.0,26758.0,19426.0,0.94,21.20,12.70,36.59,156.0,1.38,1.15,36.37,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3247.20,NaN,NaN,2.91,81180.0,46731.0,0.0,0.63,135.65,0.00,135.65,2.0,inf,0.61,24.14,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,27.50,H-SC,12.84,130423.0,5126.0,53956.0,0.54,4.09,41.37,47.15,144.0,0.10,0.98,63.17,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,19.36,H-SC,12.11,152958.0,26758.0,19426.0,0.94,21.20,12.70,36.59,156.0,1.38,1.15,36.37,XY24,NTT,HEALTHCARE
26,DABUR,505.20,735.00,-4.49,H-MC,4.94,206992.0,9964.0,79651.0,1.68,5.06,38.48,45.49,100.0,0.13,1.56,18.76,XY24,BTT,FMCG
20,CAMS,3643.00,5226.82,-5.92,H-SC,0.23,105689.0,3685.0,40659.0,-2.97,3.61,38.47,43.48,120.0,0.09,0.79,21.00,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.50,H-SC,9.75,175430.0,10760.0,57208.0,-0.02,6.53,32.61,41.28,119.0,0.19,1.32,15.04,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3247.20,NaN,NaN,2.91,81180.0,46731.0,0.0,0.63,135.65,0.00,135.65,2.0,inf,0.61,24.14,AR,ATH,ELECTRICAL
93,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,7.51,100500.0,9002.0,69747.0,-3.18,9.84,69.40,86.07,211.0,0.13,0.76,45.38,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,13.18,137583.0,14247.0,119518.0,0.41,11.55,86.87,108.45,118.0,0.12,1.03,36.49,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,27.50,H-SC,12.84,130423.0,5126.0,53956.0,0.54,4.09,41.37,47.15,144.0,0.10,0.98,63.17,MH,ATH,POWER
94,WIPRO,243.46,420.00,-9.67,M-LC,5.54,154814.0,3869.0,105583.0,-0.20,2.56,68.20,72.51,55.0,0.04,1.16,8.74,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.02,19929.0,-29664.0,865.0,-0.71,-59.81,4.34,-58.07,261.0,-34.29,0.15,33.65,XY24,ATH,MISC
74,SIS,477.00,477.00,2346.11,H-SC,6.62,65816.0,-15274.0,15276.0,0.77,-18.84,23.21,0.00,157.0,-1.00,0.49,33.39,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.18,69965.0,-43584.0,83636.0,0.66,-38.38,119.54,35.27,267.0,-0.52,0.53,71.24,XR,NTT,HOTELS
32,ENRIN,1377.95,3247.20,NaN,NaN,2.91,81180.0,46731.0,0.0,0.63,135.65,0.00,135.65,2.0,inf,0.61,24.14,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-17.80,H-SC,16.87,93225.0,-4755.0,26625.0,0.89,-4.85,28.56,22.32,151.0,-0.18,0.70,35.21,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.18,51612.0,10357.0,7076.0,-0.67,25.10,13.71,42.26,198.0,1.46,0.39,21.51,XY24,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,7.02,19929.0,-29664.0,865.0,-0.71,-59.81,4.34,-58.07,261.0,-34.29,0.15,33.65,XY24,ATH,MISC
58,MEDANTA,1087.93,1486.00,19.36,H-SC,12.11,152958.0,26758.0,19426.0,0.94,21.20,12.70,36.59,156.0,1.38,1.15,36.37,XY24,NTT,HEALTHCARE
91,VBL,492.64,672.28,-7.17,H-LC,11.92,281751.0,16711.0,79933.0,0.31,6.30,28.37,36.46,3.0,0.21,2.12,20.33,X40N,ATH,FMCG
72,SHALBY,261.39,327.00,965.39,M-SC,9.07,144837.0,-36306.0,81775.0,-0.43,-20.04,56.46,25.10,233.0,-0.44,1.09,16.07,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.80
1,25,41.75
2,50,70.75


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.03
LC    35.63
MC    22.17
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.71
X40N     14.94
X40      12.65
XY25     10.97
XR        9.06
AR        8.27
OX40N     7.85
X5K       2.23
MH        1.72
X200      1.38
SR        1.20
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.49
H-SC    27.41
H-MC    19.24
M-SC    12.16
M-LC     7.09
M-MC     2.59
L-SC     1.46
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.69,-0.20,33.73
IT,11.79,-15.77,72.81
FINANCE,8.41,-16.49,58.76
BANKS,7.47,-14.56,61.30
PAINTS,5.73,-14.44,37.00
ELECTRICAL,5.45,-1.67,50.09
MISC,5.33,-21.85,85.00
HEALTHCARE,4.69,3.19,22.52
AUTO,4.33,-17.18,68.17


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2760216.0,25
XR,1088785.0,13
AR,1044424.0,10
X40,771153.0,10
X40N,561973.0,12
XY25,479210.0,8
OX40N,332556.0,10
SR,183450.0,2
X5K,142649.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2673931.0,30
M-SC,1352944.0,15
H-LC,1207904.0,20
H-MC,1145996.0,15
M-LC,433114.0,5
M-MC,331922.0,2
L-SC,267808.0,3
L-MC,56830.0,1
L-LC,49429.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920073.0      7
M-SC       XY24      835253.0      7
H-SC       AR        611882.0      4
           XR        512564.0      6
H-LC       X40       504011.0      6
H-MC       XY24      445723.0      4
M-MC       XY24      331922.0      2
H-LC       X40N      270190.0      6
           AR        217056.0      2
H-SC       X40N      216781.0      4
H-MC       XY25      205479.0      2
           X40       192529.0      3
H-SC       SR        183450.0      2
L-SC       XR        180889.0      2
M-LC       XY24      179904.0      2
H-SC       OX40N     175225.0      6
M-SC       AR        157423.0      2
M-LC       X5K       131001.0      1
H-MC       XR        119381.0      1
M-SC       XY25      118534.0      1
           XR        113538.0      2
M-LC       XR        105583.0      1
H-LC       XY25       89142.0      3
L-SC       OX40N      86919.0      1
H-LC       X200       76457.0      1
H-MC       X40N       75002.0      2
M-SC       X40        74613.0      1
H-MC       AR         58063.0      1
L-MC       XR         56830.0      1
H-SC       MH         53956.0      1
M-SC       OX40N      53583.0      2
L-LC       XY25       49429.0      1
H-LC       XY24       39400.0      1
H-MC       MH         32990.0      1
           OX40N      16829.0      1
M-LC       XY25       16626.0      1
H-LC       X5K        11648.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
